In [ ]:
import cv2
import os
import numpy as np
import pandas as pd

# 画像が格納されているフォルダのパス
folder_path = "processed_img_all/"

# 結果を保存するフォルダの作成
draw_folder = "draw_img_2"
circle_folder = "circle_img_2"
os.makedirs(draw_folder, exist_ok=True)
os.makedirs(circle_folder, exist_ok=True)

# フォルダ内の画像ファイルを取得
image_files = os.listdir(folder_path)

def circularity(contour):
    '''
    円形度を求める

    Parameters
    ----------
    contour : ndarray
        輪郭の(x,y)座標の配列

    Returns
    -------
    円形度
    '''

    # 輪郭の点の数が2以下の場合は計算できないため、0を返す
    if len(contour) <= 2:
        return None

    # 面積
    area = cv2.contourArea(contour)
    # 周囲長
    length = cv2.arcLength(contour, True)

    # 円形度を返す
    return round(4 * np.pi * area / (length * length), 3)  # 円形度を小数第3位まで丸める

# データを格納する辞書を初期化
circularity_dict = {
    '図鑑No.': [],
    '円形度': []
}

for image_file in image_files:
    # 画像のパス
    image_path = os.path.join(folder_path, image_file)
    
    # 画像の読み込み
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)  # アルファチャンネルを含む画像を読み込む
    
    # 読み込んだ画像が正しくない場合はスキップ
    if img is None:
        continue
    
    # アルファチャンネルが存在する場合、それを考慮に入れる
    if img.shape[2] == 4:
        # アルファチャンネルを取得
        alpha_channel = img[:,:,3]
        # アルファチャンネルを二値化
        _, alpha_channel_bin = cv2.threshold(alpha_channel, 128, 255, cv2.THRESH_BINARY)
        # アルファチャンネルを考慮したグレースケール画像を作成
        gray = cv2.bitwise_and(alpha_channel_bin, alpha_channel_bin)
    else:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)  # 閾値設定を自動的に行う
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 輪郭が存在する場合にのみ処理を行う
    if contours:
        # 最も輪郭を構成する点が多い輪郭を見つける
        max_contour = max(contours, key=len)
        
        # 円形度の計算
        val = circularity(max_contour)
        if val is not None:
            # 画像ファイル名から拡張子を削除
            file_name = os.path.splitext(image_file)[0]
            circularity_dict['図鑑No.'].append(file_name)
            circularity_dict['円形度'].append(val)
            
            # 輪郭を描画
            draw_img = cv2.drawContours(img.copy(), [max_contour], -1, (0, 255, 0), 5)  # 線の太さを5に変更
            # 背景を白色に設定
            draw_img[draw_img[:,:,3]==0] = [255, 255, 255, 255]
            # 画像を保存
            cv2.imwrite(os.path.join(draw_folder, file_name + '.png'), cv2.cvtColor(draw_img, cv2.COLOR_RGBA2BGRA))  # 画像の保存方法を修正
            
            # 円形度を表示
            circle_img = cv2.putText(draw_img.copy(), f'Circularity: {val}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.imwrite(os.path.join(circle_folder, file_name + '.png'), cv2.cvtColor(circle_img, cv2.COLOR_RGBA2BGRA))  # 画像の保存方法を修正

# 辞書型データをデータフレームに変換
df = pd.DataFrame(circularity_dict)

# '図鑑No.' 列でソート
df.sort_values(by='図鑑No.', inplace=True)

# データフレームを表示
print(df)
